This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `hepqpr.qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from hepqpr.qallse.plotting import *
from hepqpr.qallse import *
from hepqpr.qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=.05, # 1% 
)

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = Qallse # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, random_seed=240834366, **dsmaker_config)

using /var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpym5hm0nr


14:28:49.332 DEBUG hepqpr.qallse.dsmaker.dsmaker: Loaded 120939 hits from /Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000.
14:28:49.368 DEBUG hepqpr.qallse.dsmaker.dsmaker: Filtered hits from barrel. Remaining hits: 65518.
14:28:49.429 DEBUG hepqpr.qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 55446.
14:28:49.474 DEBUG hepqpr.qallse.dsmaker.dsmaker: High Pt hits: 32302/2858
14:28:49.621 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_hits=2858
14:28:49.622 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_tracks=409
14:28:49.622 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_important_tracks=36
14:28:49.623 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_noise=651
14:28:49.623 DEBUG hepqpr.qallse.dsmaker.dsmaker:   random_seed=240834366
14:28:49.624 DEBUG hepqpr.qallse.dsmaker.dsmaker:   time=2019-10-11T14:28:49.621626
14:28:50.910 INFO hepqpr.qallse.dsmaker.dsmaker: Doublets (len=28187) generated in f/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpym5hm0nr/

In [4]:
with open(path + '-meta.json') as f:
    print(f.read())

{
    "num_hits": 2858,
    "num_tracks": 409,
    "num_important_tracks": 36,
    "num_noise": 651,
    "random_seed": 240834366,
    "time": "2019-10-11T14:28:49.621626",
    "params": {
        "input_path": "/Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000",
        "output_path": "/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpym5hm0nr",
        "density": 0.05,
        "high_pt_cut": 1.0,
        "double_hits_ok": false,
        "gen_doublets": true,
        "prefix": null,
        "random_seed": 240834366,
        "phi_bounds": null,
        "min_hits_per_track": 5
    }
}


# Execute the model

## Load the data

In [5]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 28187 doublets.
  Input precision (%): 0.9859, recall (%): 98.8848
    New precision (%): 0.9969


## Build the model

In [6]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
qslice = model.to_qubo()

14:28:51.539 DEBUG hepqpr.qallse.qallse: using config:
14:28:51.540 DEBUG hepqpr.qallse.qallse:     cheat: False
14:28:51.541 DEBUG hepqpr.qallse.qallse:     max_layer_span: 2
14:28:51.542 DEBUG hepqpr.qallse.qallse:     num_multiplier: -1
14:28:51.543 DEBUG hepqpr.qallse.qallse:     qplet_max_dcurv: 0.0001
14:28:51.544 DEBUG hepqpr.qallse.qallse:     qplet_max_strength: -0.2
14:28:51.545 DEBUG hepqpr.qallse.qallse:     qubo_bias_weight: 0
14:28:51.546 DEBUG hepqpr.qallse.qallse:     qubo_conflict_strength: 1
14:28:51.547 DEBUG hepqpr.qallse.qallse:     rz_power: 1
14:28:51.547 DEBUG hepqpr.qallse.qallse:     strength_bounds: None
14:28:51.548 DEBUG hepqpr.qallse.qallse:     tplet_max_curv: 0.0008
14:28:51.548 DEBUG hepqpr.qallse.qallse:     tplet_max_drz: 0.1
14:28:51.549 DEBUG hepqpr.qallse.qallse:     volayer_power: 2
14:28:51.550 DEBUG hepqpr.qallse.qallse:     xy_power: 1
14:28:51.553 DEBUG hepqpr.qallse.qallse:     xy_relative_strength: 0.5
14:28:52.256 INFO hepqpr.qallse.qallse:

Container with 4 slices
hi None
Return qubo with length 1092 and eta: 0, phi: 0
eta: 0, phi: 0
Qubo: {('79844_92098_107845', '79844_92098_107845'): 0, ('19699_35927_41886', '19699_35927_41886'): 0, ('42396_73906_87697', '42396_73906_87697'): 0, ('38560_76929_84225', '38560_76929_84225'): 0, ('79844_86354_92098', '79844_86354_92098'): 0, ('38560_45453_76929', '38560_45453_76929'): 0, ('32035_39670_78181', '32035_39670_78181'): 0, ('42396_73906_80994', '42396_73906_80994'): 0, ('28450_35169_41374', '28450_35169_41374'): 0, ('42420_73915_81012', '42420_73915_81012'): 0, ('18184_26604_33504', '18184_26604_33504'): 0, ('28424_35134_73401', '28424_35134_73401'): 0, ('80144_86625_92350', '80144_86625_92350'): 0, ('42396_80994_87697', '42396_80994_87697'): 0, ('38560_45453_84225', '38560_45453_84225'): 0, ('38613_45145_76979', '38613_45145_76979'): 0, ('28424_35134_41791', '28424_35134_41791'): 0, ('80144_92350_108298', '80144_92350_108298'): 0, ('28499_35800_73906', '28499_35800_73906'): 0, (

## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [7]:
%%time
# execute the qubo
response = model.sample_qubo_slices(Q=qslice)

im in a notebook sample {('79844_92098_107845', '79844_92098_107845'): 0, ('19699_35927_41886', '19699_35927_41886'): 0, ('42396_73906_87697', '42396_73906_87697'): 0, ('38560_76929_84225', '38560_76929_84225'): 0, ('79844_86354_92098', '79844_86354_92098'): 0, ('38560_45453_76929', '38560_45453_76929'): 0, ('32035_39670_78181', '32035_39670_78181'): 0, ('42396_73906_80994', '42396_73906_80994'): 0, ('28450_35169_41374', '28450_35169_41374'): 0, ('42420_73915_81012', '42420_73915_81012'): 0, ('18184_26604_33504', '18184_26604_33504'): 0, ('28424_35134_73401', '28424_35134_73401'): 0, ('80144_86625_92350', '80144_86625_92350'): 0, ('42396_80994_87697', '42396_80994_87697'): 0, ('38560_45453_84225', '38560_45453_84225'): 0, ('38613_45145_76979', '38613_45145_76979'): 0, ('28424_35134_41791', '28424_35134_41791'): 0, ('80144_92350_108298', '80144_92350_108298'): 0, ('28499_35800_73906', '28499_35800_73906'): 0, ('38613_45145_84270', '38613_45145_84270'): 0, ('28450_35169_73431', '28450_35

im in a notebook sample {('23952_31689_45867', '23952_31689_45867'): 0, ('23952_39137_45867', '23952_39137_45867'): 0, ('23952_31689_39137', '23952_31689_39137'): 0, ('23971_32192_39479', '23971_32192_39479'): 0, ('19843_28692_35983', '19843_28692_35983'): 0, ('19843_28692_42524', '19843_28692_42524'): 0, ('32192_39479_77859', '32192_39479_77859'): 0, ('19843_35983_42524', '19843_35983_42524'): 0, ('18530_33923_40321', '18530_33923_40321'): 0, ('33923_40321_78784', '33923_40321_78784'): 0, ('82192_88706_94629', '82192_88706_94629'): 0, ('42524_74007_81087', '42524_74007_81087'): 0, ('18453_26707_33903', '18453_26707_33903'): 0, ('33903_71593_78393', '33903_71593_78393'): 0, ('33923_40321_72075', '33923_40321_72075'): 0, ('42524_81087_87391', '42524_81087_87391'): 0, ('81087_87391_93570', '81087_87391_93570'): 0, ('42524_74007_87391', '42524_74007_87391'): 0, ('33923_72075_78784', '33923_72075_78784'): 0, ('39137_45867_77330', '39137_45867_77330'): 0, ('18530_27240_33923', '18530_27240_

im in a notebook sample {('28287_41700_73329', '28287_41700_73329'): 0, ('28364_35080_73363', '28364_35080_73363'): 0, ('28287_35637_41700', '28287_35637_41700'): 0, ('28287_35637_73329', '28287_35637_73329'): 0, ('80075_86274_92290', '80075_86274_92290'): 0, ('80100_86293_92029', '80100_86293_92029'): 0, ('80075_92290_108022', '80075_92290_108022'): 0, ('28364_41733_73363', '28364_41733_73363'): 0, ('28364_35080_41733', '28364_35080_41733'): 0, ('38927_45412_84457', '38927_45412_84457'): 0, ('38927_45412_77184', '38927_45412_77184'): 0, ('38927_77184_84457', '38927_77184_84457'): 0, ('81335_87595_109498', '81335_87595_109498'): 0, ('38945_45440_77205', '38945_45440_77205'): 0, ('38945_45440_84220', '38945_45440_84220'): 0, ('38945_77205_84220', '38945_77205_84220'): 0, ('26571_33488_71319', '26571_33488_71319'): 0, ('42852_74287_87595', '42852_74287_87595'): 0, ('82384_89235_95398', '82384_89235_95398'): 0, ('42852_74287_81335', '42852_74287_81335'): 0, ('82745_89543_95669', '82745_89

im in a notebook sample {('30159_43917_75359', '30159_43917_75359'): 0, ('30158_43366_75344', '30158_43366_75344'): 0, ('30158_37018_75344', '30158_37018_75344'): 0, ('80778_87530_93370', '80778_87530_93370'): 0, ('30159_37020_43917', '30159_37020_43917'): 0, ('80406_86863_92775', '80406_86863_92775'): 0, ('30159_37020_75359', '30159_37020_75359'): 0, ('22885_31216_38389', '22885_31216_38389'): 0, ('80778_93370_109219', '80778_93370_109219'): 0, ('80416_86532_92491', '80416_86532_92491'): 0, ('30180_37052_43942', '30180_37052_43942'): 0, ('30180_43942_75379', '30180_43942_75379'): 0, ('30180_37052_75379', '30180_37052_75379'): 0, ('22937_31253_37871', '22937_31253_37871'): 0, ('22937_37871_44524', '22937_37871_44524'): 0, ('22937_31253_44524', '22937_31253_44524'): 0, ('33992_40085_72140', '33992_40085_72140'): 0, ('82632_89468_95325', '82632_89468_95325'): 0, ('82231_88737_95011', '82231_88737_95011'): 0, ('82252_89138_95338', '82252_89138_95338'): 0, ('33995_40081_78630', '33995_4008

## Parse the results

In [8]:
# get all output doublets
all_doublets = model.process_sample(next(response.samples()))
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

14:28:55.836 INFO hepqpr.qallse.track_recreater: Found 0 conflicting doublets


# Evaluate the results

## Print statistics and scores

In [9]:
# stats about the qbsolv run
#en0 = dw.compute_energy(Q)
#en = response.record.energy[0]

#print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
#occs = response.record.num_occurrences
#print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

SCORE  -- precision (%): 100.0, recall (%): 21.561338289962826, missing: 211
          tracks found: 15, trackml score (%): 21.838167383348463


### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [10]:
iplot_results(dw, final_doublets, missings, dims=list('xy'))

In [11]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [12]:
iplot_results(dw, final_doublets, missings, dims=list('zxy'))